Imports

In [ ]:
import pandas as pd;
import os;
from google.colab import files;
import matplotlib.pyplot as plt
import seaborn as sns
from math import log
import operator
from tqdm.notebook import tqdm

In [ ]:
from google.colab import drive
# for mounting google drive to access training data from
# filepath: /content/drive/MyDrive/Colab Notebooks
drive.mount('/content/drive')

Data cleaning

In [ ]:
# norm(attr:string)
# apply zscore normalisation on field 'attr'

# continuous_bin(attr:string, bins:int)
# create bins distrubuted by percentile intervals equals to 'bins'. Does not yield specified number of bins
# if data is very skewed.
def norm(dataSet, attr):
    dataSet[attr] = (
        (dataSet[attr] - dataSet[attr].mean())/dataSet[attr].std()
    )
def continuous_bin(dataSet, attr, bins):
    intervals = 1/bins
    percentiles = dataSet[attr].quantile([i*intervals for i in range(0, bins + 1)]).unique()
    bins = list(map(lambda x : float(x), percentiles))


    dataSet[attr] = pd.cut(
        dataSet[attr], bins=bins,
        labels=bins[1:]
        )
def create_new_column(dataset):
    print('Before create_new_column:', dataset[['Support Calls', 'Tenure']].dtypes)

    # Ensure 'Support Calls' and 'Tenure' are numeric types for division
    support_calls_numeric = pd.to_numeric(dataset['Support Calls'], errors='coerce')
    tenure_numeric = pd.to_numeric(dataset['Tenure'], errors='coerce')

    # Initialize 'Support Calls rates' column with NaN, then fill based on conditions
    dataset['Support Calls rates'] = float('nan')

    # Calculate rate where tenure is positive
    dataset.loc[tenure_numeric > 0, 'Support Calls rates'] = (
        support_calls_numeric / tenure_numeric
    )
    # Set rate to 0 where tenure is zero or negative (including NaNs from conversion)
    # For NaN tenure, (tenure_numeric <= 0) will be false, keeping the rate as NaN, which is correct.
    dataset.loc[tenure_numeric <= 0, 'Support Calls rates'] = 0.0

    print('After create_new_column (relevant column dtypes):', dataset[['Support Calls rates', 'Support Calls', 'Tenure']].dtypes)

In [ ]:
# Task 1 (1)
# importing data into dataframes
not_loaded = True
while not_loaded:
    try:
        if "churn_train.csv" in os.listdir():
            churn_train = pd.read_csv("customer_churn_dataset-training-master.csv")
            churn_test = pd.read_csv("customer_churn_dataset-testing-master.csv")
            not_loaded = False
        else:
            churn_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/customer_churn_dataset-training-master.csv")
            churn_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/customer_churn_dataset-testing-master.csv")
            not_loaded = False
    except FileNotFoundError as e:
        files.upload()

In [ ]:
# Check for the non continuous attributes
for attr in churn_train.columns:
    if len(churn_train[attr].unique()) < 10:
        print(f"{attr}: {churn_train[attr].value_counts()}")

In [ ]:
gender = {"Female":0, "Male":1}
Sub_type = {"Standard":0, "Basic":1, "Premium":2}
# converted df
conv_churn_train = churn_train.copy()
conv_churn_train["Gender"] = conv_churn_train["Gender"].map(gender)
conv_churn_train["Subscription Type"] = conv_churn_train["Subscription Type"].map(Sub_type)

In [ ]:
# Task1 (2)
missing = conv_churn_train.isnull().any(axis=0).sum()
total = len(churn_train.index)
# print((missing / total)*100)
# 0.002% columns are null! There are 9 records that are empty.
# Because the missing data takes up a small portion of the training data I will delete samples with missing data

clean_churn_train = conv_churn_train.copy().dropna()
filled = clean_churn_train.isnull().any(axis=0).sum()
print(f"number of rows with nan:{missing}\nnumber of rows with nan after cleaning:{filled}")

Pre-processing

In [ ]:
# Task 1 (3)
norm(clean_churn_train, "Last Interaction")
variance = clean_churn_train['Last Interaction'].std() ** 2
mean = clean_churn_train['Last Interaction'].mean()
print(f"var{variance:.2e}, mean{mean:.2e}\nApproximately 1 and 0")

In [ ]:
# Task 1 (4)
continuous_bin(clean_churn_train, "Total Spend", 5)
print("Number of rows for each 'Total Spend' quantile:")
print(clean_churn_train['Total Spend'].value_counts().sort_index())

In [ ]:
# Task 1 (5)
Onehot = pd.get_dummies(clean_churn_train['Contract Length'], prefix='Contract Length')
clean_churn_train = pd.concat([clean_churn_train, Onehot], axis=1)
clean_churn_train.drop('Contract Length', axis=1, inplace=True)

In [ ]:
# Task 1 (6)
# Chose Support calls and Tenure to create secondary attribute Support Call Rate.
# Support calls has the highest correlation coefficient according to the matrix
# Therefore, by extension Support call rate would likely yield similar results
create_new_column(clean_churn_train)

In [ ]:
correlation_matrix = clean_churn_train.corr()

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='RdYlGn', fmt=".2f", linewidths=.5)
plt.title('Correlation Matrix of All Attributes (Red: High, Green: Low)')
plt.show()

Task 2

In [ ]:
#find the continuous columns
continuous_columns = []

for attr in clean_churn_train.columns:
    if len(clean_churn_train[attr].unique()) > 10:
        if attr != "CustomerID":
            continuous_columns.append(attr)
            print(f"{attr}")

In [ ]:
# Task 2 (1)
# most pre-processing steps have already been done in part 1.
for attr in continuous_columns:
    continuous_bin(clean_churn_train, attr, 10)

In [ ]:
def calcShannonEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    for featVec in dataSet: #the the number of unique elements and their occurance
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys(): labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries
        shannonEnt -= prob * log(prob,2) #log base 2
    return shannonEnt

def splitDataSet(dataSet, axis, value):
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]     #chop out axis used for splitting
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

def chooseBestFeatureToSplit(dataSet):
    # prepruning
    if len(dataSet) < 20:
        return None
    numFeatures = len(dataSet[0]) - 1
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0; bestFeature = -1
    for i in tqdm(range(numFeatures), desc="Choosing best feature"):
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)
        infoGain = baseEntropy - newEntropy
        if (infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
        # prepruning
        if bestInfoGain < 0.1:
            return None
    return bestFeature

def majorityCnt(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys(): classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

def createTree(dataSet,labels, max_depth=None, current_depth=10):
    classList = [example[-1] for example in dataSet]

    if classList.count(classList[0]) == len(classList):
        return classList[0]
    if len(dataSet[0]) == 1 or (max_depth is not None and current_depth >= max_depth):
        return majorityCnt(classList)

    bestFeat = chooseBestFeatureToSplit(dataSet)
    if bestFeat is None:
        return majorityCnt(classList)

    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}

    # Create a copy of labels to pass to recursive calls, excluding the current best feature's label
    subLabels = labels[:bestFeat] + labels[bestFeat+1:]

    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in tqdm(uniqueVals, desc=f"Building branch for {bestFeatLabel}"):
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value),subLabels, max_depth, current_depth + 1)
    return myTree

def classify(inputTree,featLabels,testVec):
    firstStr=list(inputTree.keys())[0]
    secondDict=inputTree[firstStr]
    featIndex=featLabels.index(firstStr)
    for key in secondDict:
        if testVec[featIndex]==key:
            if type(secondDict[key]).__name__=='dict':
                classLabel=classify(secondDict[key],featLabels,testVec)
            else:
                classLabel=secondDict[key]
    return classLabel

calcShannonEnt(dataSet):
Calculates Shannon entropy of the dataset. Measures impurity/uncertainty in class labels. Higher entropy means more mixed classes.

splitDataSet(dataSet, axis, value):
Splits dataset by a specific feature (axis) with a given value. Returns subset where feature equals value, removing that feature column.

chooseBestFeatureToSplit(dataSet):
Finds feature with highest information gain. Compares entropy reduction for all features. Returns index of best splitting feature.

majorityCnt(classList):
Determines most frequent class in a list. Used when no more splits are possible or for tie-breaking.

createTree(dataSet, labels):
Recursively builds decision tree using ID3 algorithm. Stops when pure classes or no features remain. Returns tree structure.

classify(inputTree, featLabels, testVec):
Traverses decision tree to classify a test sample. Follows feature branches based on test values until reaching leaf node.

I chose to perform pre-pruning to avoid overfitting during the induction of a decision tree

pre-pruning halting tree construction early, it prevents the model from developing too many branches that may merely reflect anomalies, noise, or outliers in the training data

within chooseBestFeatureToSplit i set two threshold for halting.
When the size of the split dataset is less than 20.
When the decrease in entropy\(variance\) is less than 0.1.

In [ ]:
values = clean_churn_train.values.tolist()
columns = clean_churn_train.columns.tolist()

In [ ]:
print(len(values))

In [ ]:
Decision_Tree = createTree(values[:1000], columns)

# Task
Preprocess the `churn_test` DataFrame by applying the same cleaning and feature engineering steps as done for the training data: map 'Gender' and 'Subscription Type' using existing dictionaries, drop rows with missing values, create a 'Support call rate' feature, normalize 'Last Interaction' using Z-score, apply continuous binning with 5 bins to 'Total Spend', perform one-hot encoding on 'Contract Length', and apply continuous binning with 10 bins to 'Age', 'Tenure', 'Usage Frequency', 'Support Calls', 'Payment Delay', 'Last Interaction', and 'Support call rate', then display the shape, head, and missing value counts of the resulting `clean_churn_test` DataFrame.


train pre-processsing


In [ ]:
clean_churn_test = churn_test.copy()

In [ ]:
clean_churn_test["Gender"] = clean_churn_test["Gender"].map(gender)
clean_churn_test["Subscription Type"] = clean_churn_test["Subscription Type"].map(Sub_type)

In [ ]:
missing_before = clean_churn_test.isnull().any(axis=1).sum()
clean_churn_test.dropna(inplace=True)
missing_after = clean_churn_test.isnull().any(axis=1).sum()

In [ ]:
create_new_column(clean_churn_test)

In [ ]:
norm(clean_churn_test, "Last Interaction")

**Reasoning**:
To maintain consistency with the preprocessing applied to the training data, the 'Total Spend' column in `clean_churn_test` needs to be binned into 5 continuous intervals using the `continuous_bin` function.



In [ ]:
continuous_bin(clean_churn_test, "Total Spend", 5)

In [ ]:
Onehot_test = pd.get_dummies(clean_churn_test['Contract Length'], prefix='Contract Length')
clean_churn_test = pd.concat([clean_churn_test, Onehot_test], axis=1)
clean_churn_test.drop('Contract Length', axis=1, inplace=True)

In [ ]:
for attr in continuous_columns:
    continuous_bin(clean_churn_test, attr, 10)

In [ ]:
print("Shape of clean_churn_test:", clean_churn_test.shape)
print("\nHead of clean_churn_test:")
print(clean_churn_test.head())
print("\nMissing values in clean_churn_test:")
print(clean_churn_test.isnull().sum())

In [ ]:
print(clean_churn_test.isnull().head())

**Reasoning**:
The previous output revealed that `clean_churn_test` still contains missing values in several columns after all preprocessing steps, contradicting the intention to drop all rows with missing values. To fully comply with the task's requirement and ensure a clean dataset for subsequent steps, I need to explicitly drop these remaining rows with missing values, and then re-display the requested information.



In [ ]:
print(f"Shape of clean_churn_test before final dropna: {clean_churn_test.shape}")
clean_churn_test.dropna(inplace=True)
print(f"Shape of clean_churn_test after final dropna: {clean_churn_test.shape}")
print("\nHead of clean_churn_test after final dropna:")
print(clean_churn_test.head())
print("\nMissing values in clean_churn_test after final dropna:")
print(clean_churn_test.isnull().sum())